In [1]:
import pandas as pd
import requests
import time

# 1. MACD

In [3]:
def MACD(tradePrice):
    exp12 = tradePrice.ewm(span=12, adjust=False).mean()
    exp26 = tradePrice.ewm(span=26, adjust=False).mean()
    macd = exp12 - exp26
    exp= macd.ewm(span=9, adjust=False).mean()
    return exp

In [7]:
if __name__ == "__main__":
    url = "https://api.upbit.com/v1/candles/days"
    querystring = {"market":"KRW-BTC", "count":"200"}

    response = requests.request("GET",url,params=querystring)
    data = response.json()

    df = pd.DataFrame(data)
    df = df.iloc[::-1]

    macd = MACD(df['trade_price'])
    print(macd[0])

1002125.730514349


# 2. RSI

In [10]:
def rsi(ohlc: pd.DataFrame, period: int = 14):
    ohlc['trade_price'] = ohlc['trade_price']
    # delta 함수에 차분을 계산하여 저장한다.
    delta = ohlc['trade_price'].diff()
    gains, declines = delta.copy(), delta.copy()
    gains[gains < 0] = 0
    declines[declines > 0] = 0
    _gain = gains.ewm(com=(period - 1), min_periods=period).mean()
    _loss = declines.abs().ewm(com=(period - 1), min_periods=period).mean()

    RS = _gain / _loss
    return pd.Series(100 - (100 / (1 + RS)), name="RSI")

In [12]:
url = "https://api.upbit.com/v1/candles/minutes/60"
querystring = {"market":"KRW-BTC", "count":"200"}
response = requests.request("GET", url, params=querystring)
data = response.json()
df = pd.DataFrame(data)
df = df.reindex(index=df.index[::-1]).reset_index()
nrsi = rsi(df,14).iloc[-1]
print('현재 rsi : ' + str(nrsi))



현재 rsi : 55.99173771320481
